# Using the PowerGrid Models API

---

## Introduction to the PowerGrid Model API


The PowerGrid Models API is used to pull model information from the Blazegraph Database, inlcuding the names, mRIDs, measurements, and nominal values of power system equipment in the feeder (such as lines, loads, switches, transformers, and DERs).

In the Application Components diagram (explained in detail with sample code in [GridAPPS-D Application Structure](../overview/2.4-GridAPPS-D-Application-Structure.ipynb)), the PowerGrid Models API is used for querying for the power system model and querying for model measurement MRIDs.

![power grid models](images/3.3/power_grid_models_usage.png)

---

## API Syntax Overview

__Application passes query to GridAPPS-D Platform__

First, the application creates a query message for requesting information about the desired power system components in the format of a JSON string or equivalant Python dictionary object. The syntax of this message is explained in detail below.

The query is sent using `gapps.get_response(topic, message)` with a response expected back from the platform within the specified timeout period.

The application then passes the query through the PowerGrid Models API to the GridAPPS-D Platform, which publishes it to the `goss.gridappsd.process.request.data.powergridmodel` queue channel on the GOSS Message Bus. If the app is authenticated and authorized to pass queries, the query message is delivered to the data managers, which obtain the desired information from the Blazegraph Database. 

__GridAPPS-D Platform responds to Application query__

The data managers then publish the response from the Blazegraph Database to the appropriate queue channel. The PowerGrid Models API then returns the desired information back to the application as a JSON message or equivalant Python dictionary object.

### API Communication Channel

All queries passed to the PowerGrid Models API need to use the correct communication channel, which is obtained using the [GridAPPS-D Topics library](3.1-API-Communication-Channels.ipynb). 

    The PowerGrid Model API uses a `/queue/` channel to pull power system model info from the the Blazegraph Database. The base static string used is `goss.gridappsd.process.request.data.powergridmodel`, which can be called using the `.REQUEST_POWERGRID_DATA` or `.BLAZEGRAPH` methods from the topics library.

    When developing in python, it is recommended to use the `.REQUEST_POWERGRID_DATA` method. When using the STOMP client in GridAPPS-D VIZ, it is necessary to use the base static string.

In [ ]:
from gridappsd import topics as t
topic = t.REQUEST_POWERGRID_DATA

---

### Structure of a Query Message

Queries passed to PowerGrid Models API are formatted as python dictionaries or equivalent JSON scripts wrapped as a python string. 

```
message = {
    "requestType": "INSERT QUERY HERE",
    "resultFormat": "JSON",
    "modelId": "OPTIONAL INSERT MODEL mRID HERE",
    "objectType": "OPTIONAL INSERT CIM CLASS HERE",
    "objectId": "OPTIONAL INSERT OBJECT mRID HERE",
    "filter": "OPTIONAL INSERT SPARQL FILTER HERE"
}
```

The components of the message are as follows:

* `"requestType":` -- Specifies the type of query. Available requestType are listed in the next section.


* `"resultFormat":` -- Specifies the format of the response, can be `"JSON"`, `"CSV"`, or `"XML"`. (CAUTION: the PowerGridModel API uses the key _resultFormat_, while the Timeseries API uses the key _reponseFormat_. Using the wrong key for either API will result in a java.lang error.)


* `"modelID":` -- Optional. Used to filter the query to only one particular model whose mRID is specified. Be aware of spelling and capitalization differences between JSON query spelling `"modelId"` and Python Library spelling `model_id`.


* `"objectType":` -- Optional. Used to filter the query to only one CIM class of equipment. Speciying the _objectID_ will override any values specified for _objectType_. 


* `"objectID":` -- Optional. Used to filter the query to only one object whose mRID is specified. Specifying the _objectID_ will override any values specified for _objectType_. 


* `"filter":` -- Optional. Used to filter the query using a SPARQL filter. SPARQL queries are covered in the next lesson.


The usage of each of these message components are explained in detail with code block examples below. 

__Important__: Be sure to pay attention to placement of commas ( __,__ ) at the end of each JSON line. Commas are placed at the end of each line _except_ the last line. Incorrect comma placement will result in a JsonSyntaxException. 

All of the queries are passed to the PowerGrid Model API using the `.get_response(topic, message)` method for the GridAPPS-D platform connection variable.

---

### Specifying the requestType

Below are the possible `requestType` strings that are used to specify the type of each query. Executable code block examples are provided for each of the requests in the subsections below.

The first group of _requestType_ key-value pairs are for queries for information related to the just the mRIDs of a set of feeders or set of equipment within a particular feeder:

* `"requestType": "QUERY_MODEL_NAMES"` -- [Query for the list of all model name mRIDs](#Query-for-mRIDs-of-all-Models)


* `"requestType": "QUERY_OBJECT_IDS"` -- [Query for a list of all mRIDs for objects of a CIM class](#Query-for-mRIDs-of-Objects-in-a-Feeder)

The second group of _requestType_ key-value pairs are for queries for Python dictionaries containing all specifics of a set of feeders or set of equipment within a particular feeder:

* `"requestType": "QUERY_MODEL_INFO"` -- [Query for the dictionary of all details for all feeders in Blazegraph](#Query-for-Details-Dictionary-of-all-Models)


* `"requestType": "QUERY_OBJECT_DICT"` -- [Query for the dictionary of all details for an object using either its _objectType_ OR its _objectID_](#Query-for-Object-Dictionary)

The third group of _requestType_ key-value pairs are for obtaining information about CIM objects and attributes:

* `"requestType": "QUERY_OBJECT_TYPES"` -- [Query for the types of CIM classes of objects in the model](#Query-for-CIM-Classes-of-Objects-in-Model)


* `"requestType": "QUERY_OBJECT"` -- [Query for CIM attributes of an object using its unique mRID](#Query-for-CIM-Attributes-of-an-Object)


One of the most important queries is for object measurements. Each piece of equipment has voltage, power, and/or position measurements associated with it. Each measurement has a unique mRID which is different from that of the equipment.

* `"requestType": "QUERY_OBJECT_MEASUREMENTS"` -- [Query for all measurement types and mRIDs for an object using either its _objectType_ OR its _ObjectID_](#Querying-for-Measurements)

The last group of _requestType_ key-value pairs are for queries based on SPARQL filters or complete SPARQL queries. Usage of these two _requestType_ is made in conjunction with custom SPARQL queries given in [Sample SPARQL Queries](https://github.com/GRIDAPPSD/CIMHub/blob/master/queries.txt).

* `"requestType": "QUERY_MODEL"` -- Query for all  part of a specified model, filtered by object type using a SPARQL filter.


* `"requestType": "QUERY"` -- Query using a complete SPARQL query.

---

### CIM Objects Supported by PowerGrid Models API

Below is a list of CIM object classes that can be queried for using the PowerGrid Models API. Other classes and associated attributes need to be queried for using a custom SPARQL query. Sample SPARQL queries for must custom queries can be found in the [CIMHub queries.txt file](https://github.com/GRIDAPPSD/CIMHub/blob/master/queries.txt)

__CIM Classes supported by the PowerGrid Models API__

* `ACLineSegment`
* `Breaker`
* `ConnectivityNode`
* `EnergyConsumer`
* `EnergySource`
* `Fuse`
* `LinearShuntCompensator`
* `LoadBreakSwitch`
* `PowerElectronicsConnection`
* `PowerTransformer`
* `Recloser`
* `SynchronousMachine`
* `TransformerTank`

__CIM Classes requiring custom SPARQL queries__

* `ACLineSegmentPhase`
* `Analog`
* `Asset`
* `BaseVoltage`
* `BatteryUnit`
* `ConcentricNeutralCableInfo`
* `CoordinateSystem`
* `CurrentLimit`
* `Discrete`
* `EnergyConsumerPhase`
* `Feeder`
* `GeographicalRegion`
* `House`
* `IEC61970CIMVersion`
* `LinearShuntCompensatorPhase`
* `LoadResponseCharacteristic`
* `Location`
* `NoLoadTest`
* `OperationalLimitSet`
* `OperationalLimitType`
* `OverheadWireInfo`
* `PerLengthPhaseImpedance`
* `PerLengthSequenceImpedance`
* `PhaseImpedanceData`
* `PhotovoltaicUnit`
* `PositionPoint`
* `PowerElectronicsConnectionPhase`
* `PowerTransformerEnd`
* `PowerTransformerInfo`
* `RatioTapChanger`
* `RegulatingControl`
* `ShortCircuitTest`
* `SubGeographicalRegion`
* `Substation`
* `SwitchPhase`
* `TapChangerControl`
* `TapChangerInfo`
* `TapeShieldCableInfo`
* `Terminal`
* `TopologicalIsland`
* `TopologicalNode`
* `TransformerCoreAdmittance`
* `TransformerEndInfo`
* `TransformerMeshImpedance`
* `TransformerTankEnd`
* `TransformerTankInfo`
* `VoltageLimit`
* `WirePosition`
* `WireSpacingInfo`


---

## Querying for Model mRIDS

Every piece of equipment has a unique mRID, as explained in [Intro to Common Information Model](../overview/2.6-Common-Information-Model.ipynb). These mRIDs are used to identify and communicate with equipment in GridAPPS-D. 

The set of queries below provide just the mRIDs of equipment matching the query filters. If the full details of equipment are desired (e.g. name and properties), use the [Query for Equipment Dictionaries](#Querying-for-Equipment-Dictionaries) API calls in the next section. 

This section outlines the pre-built JSON queries that can be passed to the PowerGrid Model API to obtain mRIDs and other information for all models and feeders stored in the Blazegraph Database.

### Query for mRIDs of all Models

This query obtains a list of all the model MRIDs stored in the Blazegraph database. 

Query requestType:

* `"requestType": "QUERY_MODEL_NAMES"`

Allowed parameters:

* `"resultFormat":` – "XML" / "JSON" / "CSV" -- Optional. Will return results as a list in the format selected. 

In [ ]:
message = {
    "requestType": "QUERY_MODEL_NAMES",
    "resultFormat": "JSON"
}

gapps.get_response(topic, message)

The same query can be passed through the STOMP client by specifying the `goss.gridappsd.process.request.data.powergridmodel` topic and the same message without the python wrapping:

![query-object-ids](images/3.3/QUERY_MODEL_NAMES.png)

__Python Library Method__

The GridAPPSD-Python library contains a pre-built method for obtaining the mRIDs of all models, providing identical results to those obtained above.

The `.query_model_names` method is associated with the GridAPPSD connection object and returns a list of all the CIM XML classes of objects present in the Blazegraph for a particular model or all models in the database.

In [ ]:
gapps.query_model_names()

---

### Query for mRIDs of Objects in a Feeder

This query is used to obtain all the mRIDs of objects of a particular CIM class in the feeder. 

Query responseType is

* `"requestType": "QUERY_OBJECT_IDS"`

Allowed parameters are:

* `"modelId":` "model name mRID" -- When specified it searches against that model, if empty it will search against all models
* `"objectType":` "CIM Class" -- Optional. Specifies the type of objects you wish to return details for.
* `"resultFormat":` – "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected. 

Within a particular feeder, it is possible to query for objects of all the CIM classes supported by PowerGrid Models API (discussed above in [CIM Objects Supported by the API](#CIM-Objects-Supported-by-PowerGrid-Models-API)). Other types of equipment require custom SPARQL queries.

Note that the RDF URI is not included in the query, only the name of the class, such as `"objectType": "ACLineSegment"` or `"objectType": "LoadBreakSwitch"`.

In [ ]:
message = {
    "requestType": "QUERY_OBJECT_IDS",
    "modelId": model_mrid,
    "objectType": "LoadBreakSwitch",
    "resultFormat": "JSON"
} 

gapps.get_response(topic, message)

It is possible to then filter the response to just a list of the mRIDs:

In [ ]:
response_obj = gapps.get_response(topic, message)
switch_mrids = response_obj['data']['objectIds']
print(switch_mrids)

The same query can be passed through the STOMP client by specifying the `goss.gridappsd.process.request.data.powergridmodel` topic and the same message without the python wrapping:

![query-object-ids](images/3.3/QUERY_OBJECT_IDS.png)

---

## Querying for Equipment Dictionaries

This section outlines the pre-built JSON queries that can be passed to the PowerGrid Model API to obtain mRIDs and other information for a particular object or a class of objects for one or more feeders stored in the Blazegraph Database.

All of the examples in this section use the IEEE 13 node model.

In [ ]:
model_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62" # IEEE 13 Node used for all example queries

### Query for Dictionary of all Models

This query returns a list of names and MRIDs for all models, substations, subregions, and regions for all available feeders stored in the Blazegraph database.

Query requestType:

* `"requestType": "QUERY_MODEL_INFO"`

Allowed parameters:

* `"resultFormat":` – "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected.

In [ ]:
message = {
    "requestType": "QUERY_MODEL_INFO",
    "resultFormat": "JSON"
}

In [ ]:
gapps.get_response(topic, message)

The same query can be passed through the STOMP client by specifying the `goss.gridappsd.process.request.data.powergridmodel` topic and the same message without the python wrapping:

![query-model-info](images/3.3/QUERY_MODEL_INFO.png)

---

### Query for Object Dictionary

This query returns a python dictionary of all the equipment attributes and mRIDs. The query can be for 1) all objects of a particular `objectType` or 2) for those connected to a particular object based on the `objectId`. 

If neither `objectType` or `objectId` is provided, the query will provide all measurements belonging to the model.
Query responseType is

* `"requestType": "QUERY_OBJECT_DICT"`

Allowed parameters are:

* `"modelId":` "model name mRID" -- When specified it searches against that model, if empty it will search against all models

* `"objectId":` "object mRID" -- Optional. Specifies the type of objects you wish to return details for.

* `"objectType":` "CIM Class" -- Optional. Specifies the type of objects you wish to return details for.

* `"resultFormat":` "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected.

Speciying the `objectID` will override any values specified for `objectType`.

__Example 1: Querying for model dictionary for an `"objectID":`__

In [ ]:
message = {
    "requestType": "QUERY_OBJECT_DICT",
    "modelId": model_mrid,
    "objectId": switch_mrids[1],
    "resultFormat": "JSON"
}

gapps.get_response(topic, message)

The same query can be passed through the STOMP client by specifying the `goss.gridappsd.process.request.data.powergridmodel` topic and the same message without the python wrapping:

![query-object-dict](images/3.3/QUERY_OBJECT_DICT_2.png)

__Example 2: Querying for model dictionary for an `"objectType":`__

In [ ]:
message = {
    "requestType": "QUERY_OBJECT_DICT",
    "modelId": model_mrid,
    "objectType": "TransformerTank",
    "resultFormat": "JSON"
}

gapps.get_response(topic, message)

The same query can be passed through the STOMP client by specifying the `goss.gridappsd.process.request.data.powergridmodel` topic and the same message without the python wrapping:

![query-object-dict](images/3.3/QUERY_OBJECT_DICT.png)

---

## Querying for CIM Attributes



### Query for CIM Classes of Objects in Model

This query is used to query for a list of all the CIM XML classes of objects present in the Blazegraph for a particular model or all models in the database.

Query requestType is

* `"requestType": "QUERY_OBJECT_TYPES"`

Allowed parameters are

* `"modelId":` "model name mRID" -- Optional. Searches only the particular model identified by the given unique mRID
* `"resultFormat":` – "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected. 

__1) Query entire Blazegraph database__

Omit the `"modelId"` parameter to search the entire blazegraph database. 

In [ ]:
message = {
    "requestType": "QUERY_OBJECT_TYPES",
    "resultFormat": "JSON"
}

In [ ]:
gapps.get_response(topic, message)

The same query can be passed through the STOMP client by specifying the `goss.gridappsd.process.request.data.powergridmodel` topic and the same message without the python wrapping:

![query-object-types](images/3.3/QUERY_OBJECT_TYPES.png)

__2) Query for only a particular model__

Specify the model MRID as a python string and pass it as a parameter to the method to return only the CIM classes of objects in that particular model. 

Be aware of spelling and capitalization differences between JSON query spelling `"modelId"` and Python Library spelling `model_id`.

In [ ]:
message = {
    "requestType": "QUERY_OBJECT_TYPES",
    "modelId": model_mrid,
    "resultFormat": "JSON"
}

In [ ]:
gapps.get_response(topic, message)

The same query can be passed through the STOMP client by specifying the `goss.gridappsd.process.request.data.powergridmodel` topic and the same message without the python wrapping:

![query-object-types-2](images/3.3/QUERY_OBJECT_TYPES_2.png)

__Python Library Method__

The GridAPPSD-Python library contains a pre-built method for obtaining all the CIM classes in the model, providing identical results to those obtained above.

The `.query_object_types` method is associated with the GridAPPSD connection object and returns a list of all the CIM XML classes of objects present in the Blazegraph for a particular model or all models in the database.

Allowed parameters are

* model_id (optional) - when specified, it searches only the particular model identified by the given unique mRID

__1) Query entire Blazegraph database__

Leave the arguments blank to search all models in the Blazegraph database

In [ ]:
gapps.query_object_types()

__2) Query for only a particular model__

Specify the model MRID as a python string and pass it as a parameter to the method to return only the CIM classes of objects in that particular model

In [ ]:
gapps.query_object_types(model_mrid)

---

### Query for CIM Attributes of an Object

This query is used to obtain all the attributes and mRIDs of those attributes for a particular object whose mRID is specified. 

Query responseType is

* `"requestType": "QUERY_OBJECT"`

Allowed parameters are:

* `"modelId":` "model name mRID" -- When specified it searches against that model, if empty it will search against all models
* `"objectId":` "object mRID" -- Optional. Specifies the type of objects you wish to return details for.
* `"resultFormat":` – "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected.


In [ ]:
object_mrid = "_2858B6C2-0886-4269-884C-06FA8B887319"

message = {
    "requestType": "QUERY_OBJECT",
    "resultFormat": "JSON",
    "modelId": model_mrid,
    "objectId": object_mrid
}

gapps.get_response(topic, message)

The same query can be passed through the STOMP client by specifying the `goss.gridappsd.process.request.data.powergridmodel` topic and the same message without the python wrapping:

![query-object](images/3.3/QUERY_OBJECT.png)

__Python Library Method__

The GridAPPSD-Python library contains a pre-built method for obtaining the mRIDs of all models, providing identical results to those obtained above.

The `.query_object` method is associated with the GridAPPSD connection object and returns a list of all the CIM XML classes of objects present in the Blazegraph for a particular model or all models in the database.

In [ ]:
model_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62" # IEEE 13 Node used for all example queries
object_mrid = "_2858B6C2-0886-4269-884C-06FA8B887319"

gapps.query_object(model_mrid, object_mrid)

---

## Querying for Object Measurements

### Object mRIDs vs Measurement mRIDs

A key concept in GridAPPS-D and CIM XML power system models is the difference between the object mRID of a piece of equipment and multiple measurement mRIDs associated with its control settings and power flow values.

Measurements differ from the state variables (e.g. those obtained from State Estimator or a power flow calculation) in that the values are measured here and not calculated or estimated. Each Measurement is associated to a _PowerSystemResource_, and in GridAPPS-D for now, also a Terminal that belongs to the same _PowerSystemResource_. (Non-electrical measurements, for example weather, would not have the Terminal). 

The _measurementType_ is a string code from IEC 61850, with the following currently suppported:

* __PNV__ -- Phase to Neutral Voltage


* __VA__  -- Volt-Amperes (apparent power)


* __A__ -- Amperes (current)


* __POS__ -- Position for switches and transformer taps

Each measurement object has a __name__, __mRID__, and __phases__. In GridAPPS-D, each phase is measured individually so multi-phase codes like ABC should not be used. 

Pos measurements will be discrete, for such things as tap position, switch position, or capacitor bank position. 

The others will be Analog, with magnitude and optional angle in degrees. 

Each MeasurementValue will have a timeStamp and mRID inherited from IdentifiedObject, so the values can be traced.



### Querying for Measurements

This query returns details for the measurements within a model. The query can be for 1) all objects of a particular `objectType` or 2) for those connected to a particular object based on the `objectId`. 

If neither `objectType` or `objectId` is provided, the query will provide all measurements belonging to the model.
Query responseType is

* `"requestType": "QUERY_OBJECT_MEASUREMENTS"`

Allowed parameters are:

* `"modelId":` "model name mRID" -- When specified it searches against that model, if empty it will search against all models

* `"objectId":` "object mRID" -- Optional. Specifies the type of objects you wish to return details for.

* `"objectType":` "CIM Class" -- Optional. Specifies the type of objects you wish to return details for.

* `"resultFormat":` "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected.

Speciying the `objectID` will override any values specified for `objectType`.

__Example 1: Querying for all measurements for an `"objectID":`__

In [ ]:
message = {
    "requestType": "QUERY_OBJECT_MEASUREMENTS",
    "modelId": model_mrid,
    "objectId": switch_mrids[1],
    "resultFormat": "JSON"
}

gapps.get_response(topic, message)

The same query can be passed through the STOMP client by specifying the `goss.gridappsd.process.request.data.powergridmodel` topic and the same message without the python wrapping:

![query-object](images/3.3/QUERY_OBJECT_MEASUREMENTS.png)

__Example 2: Querying for all measurements for an `"objectType":`__

In [ ]:
message = {
    "requestType": "QUERY_OBJECT_MEASUREMENTS",
    "modelId": model_mrid,
    "objectType": "ACLineSegment",
    "resultFormat": "JSON"
}

gapps.get_response(topic, message)

The same query can be passed through the STOMP client by specifying the `goss.gridappsd.process.request.data.powergridmodel` topic and the same message without the python wrapping:

![query-object](images/3.3/QUERY_OBJECT_MEASUREMENTS_2.png)

### Filtering Returned Data

After receiving the python dictionary of measurements, it will be necessary to parse it to inlcude just the desired set of measurements. This is done using the method presented in [Parsing Returned Data](3.2-API-Message-Structure.ipynb#3.-Parsing-Returned-Data)

In [ ]:
message = {
    "modelId": model_mrid,
    "requestType": "QUERY_OBJECT_MEASUREMENTS",
    "resultFormat": "JSON",
    "objectType": "LoadBreakSwitch"
}

obj_msr_ACline = gapps.get_response(topic, message, timeout=10)

# Filter to just values for 'data' key
obj_msr_ACline = obj_msr_ACline['data'] 

# Chose specific measurement mrid. Screen out those whose type is not PNV. For example,
obj_msr_ACline = [k for k in obj_msr_ACline if k['type'] == 'Pos']

obj_msr_ACline

---

## Querying with a Custom SPARQL String

This section outlines how the PowerGrid Models API can be used to pass custom SPARQL queries for CIM objects and properties that do not have pre-built JSON string queries.

### Query using a SPARQL filter

This query returns a dictionary of all objects matching the particular filter. The query can be for 1) all objects of a particular `objectType` or 2) those with attributes corresponding to a particular SPARQL filter, or both. 

If neither `objectType` or `filter` is specified, the query will provide all objects belonging to the model.


Query responseType is

* `"requestType": "QUERY_MODEL"`

Allowed parameters are:

* `"modelId":` "model name mRID" -- Optional. When specified it searches against that model, if empty it will search against all models

* `"objectType":` "CIM Class" -- Optional. Specifies the type of objects you wish to return details for.

* `"filter":` "SPARQL triple" -- Optional. Applies the SPARQL triple filter to the query results

* `"resultFormat":` "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected.


In [ ]:
message = {
        "requestType": "QUERY_MODEL",
        "modelId": model_mrid,
        "resultFormat": "JSON",
        "filter": "?s cim:IdentifiedObject.name '650z'",
        "objectType": "http://iec.ch/TC57/CIM100#ConnectivityNode"
}

gapps.get_response(topic, message)

The same query can be passed through the STOMP client by specifying the `goss.gridappsd.process.request.data.powergridmodel` topic and the same message without the python wrapping:

![query-model](images/3.3/QUERY_MODEL.png)

### Query using a Generic SPARQL Query

This query is used to pass a generic SPARQL query to the Blazegraph database. 

Query responseType is

* `"requestType": "QUERY"`


Allowed parameters are:

* `"modelId":` "model name mRID" -- Optinal. When specified it searches against that model, if empty it will search against all models

* `"queryString":` "SPARQL query text" -- Applies the SPARQL triple filter to the query results

* `"resultFormat":` "XML" / "JSON" / "CSV" -- Will return results as a list in the format selected.


In [ ]:
message = {
    "requestType": "QUERY",
    "queryString": "select ?feeder_name ?subregion_name ?region_name WHERE {?line r:type c:Feeder.?line c:IdentifiedObject.name  ?feeder_name.?line c:Feeder.NormalEnergizingSubstation ?substation.?substation r:type c:Substation.?substation c:Substation.Region ?subregion.?subregion  c:IdentifiedObject.name  ?subregion_name .?subregion c:SubGeographicalRegion.Region  ?region . ?region   c:IdentifiedObject.name  ?region_name}",
    "resultFormat": "JSON"
}

gapps.get_response(topic, message)

Although it is possible to check these queries using the STOMP client, it is recommended to validate generic SPARQL queries using the Blazegraph Workbench hosted on [localhost:8889/bigdata/#query](localhost:8889/bigdata/#query). The STOMP client contains a JSON validator which may struggle with the line breaks included in SPARQL queries.

---

## Available Models in Default Installation

The GridAPPS-D Platform comes by default loaded with several distribution feeders ranging in size from 13 to 9500 nodes. Each feeder and recommended usage cases are summarized below.

### IEEE 13 Node Model

This is a very small distribution test feeder operating at 4.16 kV voltage level. It consists of a single voltage regulator at the substation, overhead and underground lines, shunt capacitor, and an in-line transformer.  This feeder is relatively highly loaded and provides a good test of the convergence of the problem for a very unbalanced system.

This model is recommended for debugging as the model is small enough that issues can be traced by hand.



### IEEE 123 Node Model

This models a medium-sized unbalanced distribution system operating at the nominal voltage of 4.16 kV. It consists of overhead and underground lines with single, two and three-phase laterals, along with step regulators and shunt capacitors for voltage regulation. The feeder model is characterized by the unbalanced loading having all combinations of load types (constant current, impedance, and power). It also includes a few switches to allow for the alternate paths for the power flow via feeder reconfiguration.

This model is recommended for initial app testing and debugging thorugh the first stages of development.



### IEEE 123 Node Model with PV

This model is a derivative of the IEEE 123 Node model and includes rooftop solar throughout the system.

### IEEE 8500 Node Model

This is a relatively large and realistic radial distribution feeder consisting of MV and LV (secondary) circuits [21]. Unlike other test systems, this feeder also includes 120/240V center-tapped transformers that are commonly deployed in North American power distribution systems. Thus, it allows for users to interchange between the two versions of loading conditions: balanced (208 V) and unbalanced (120 V) in the secondary transformers. Voltage control is possible using a substation LTC transformer, as well as multiple poletop regulators and capacitor banks. The feeder was created to test scalability and convergence of power flow algorithms on a large unbalanced power distribution system.

* Length: 170 km
* Nominal voltage: 12.47 kV, 120/240V
* Topology: radial
* Service transformers: yes
* Customers: 1177
* Peak load: 11.1 MW
* Normally-open switches: no

### 9500 Node Test System

The 9500 Node Test System includes three radial distribution feeders with just over 12 MW of load, consisting of both medium voltage and low voltage equipment each supplied by a different distribution substation. The three distribution feeders are connected to each other through Normally-Open switches, which is representative of the way many utilities operate in North America. One feeder represents today’s grid with low penetration of customer-side renewables. The second represents a potential future grid with microgrids and 100% renewable penetration. The third has no customer resources, a district steam plant, and a utility-scale PV farm. All three feeders have customers connected by low-voltage secondary triplex lines. 

This is the recommended target feeder for development of all new applications

![9500-node-DGs](images/3.3/9500-node-DGs.png)

### PNNL Taxonomy Feeder

### EPRI J1 Feeder

### UAF Microgrid

## Adding New Models to GridAPPS-D





* `git clone https://github.com/GRIDAPPSD/CIMHub.git`

* `pip install SPARQLWrapper numpy pandas`

Copy CIM XML file and UUID file into CIMHub directory. 

Open CIMHub directory in terminal:

* `cd CIMHub`

View the list of feeder models currently in the Blazegraph Database:

* `python3 ../CIMHub/utils/ListFeeders.py` 

Upload your new model using `curl` to Blazegraph, specifying the name and MRID of your new feeder:

* `curl -s -D- -H 'Content-Type: application/xml' --upload-file 'yournewmodel.xml' -X POST 'http://localhost:8889/bigdata/namespace/kb/sparql'`

Create the set of txt files containing the measurable objects in your new model using the `ListMeasurables` script:

* `python3 ../CIMHub/utils/ListMeasureables.py cimhubconfig.json yournewmodel _YOUR-NEW-MODEL-FEEDER-MRID-123ABC456`

Insert the measurements into Blazegraph using the `InsertMeasurements` script. The measurement MRIDs will be saved into the file `uuidfile.json`:

* ```for f in `ls -1 *txt`; do python3 ../CIMHub/utils/InsertMeasurements.py cimhubconfig.json $f uuidfile.json; done```

Insert houses into Blazegraph using the `InsertHouses` script. The measurement MRIDs for the houses will be saved into the file `uuidfile2.json`:
* `python3 ../CIMHub/utils/InsertHouses.py cimhubconfig.json _YOUR-NEW-MODEL-FEEDER-MRID-123ABC456 1 1 uuidfile2.json`


Open the GridAPPS-D Viz in a new window and you should be able to start a simulation using your new model


## Adding New Models to the PowerGrid Models GitHub Repo

All models included by default in GridAPPS-D are stored in the [PowerGrid Models GitHub repository](https://github.com/GRIDAPPSD/Powergrid-Models/)



---

![GridAPPS-D_narrow.png](../images/GridAPPS-D_narrow.png)